In [16]:
import cv2
import mediapipe as mp
import utils
import pyautogui

In [17]:
screen_width, screen_height = pyautogui.size()


mpHands=mp.solutions.hands
hands=mpHands.Hands(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    max_num_hands=1
)

In [18]:
class SmoothMouse:
    def __init__(self, alpha=0.2):
        self.alpha = alpha
        self.prev_x = None
        self.prev_y = None

    def smooth(self, x, y):
        if self.prev_x is None or self.prev_y is None:
            self.prev_x, self.prev_y = x, y
        else:
            self.prev_x = self.alpha * x + (1 - self.alpha) * self.prev_x
            self.prev_y = self.alpha * y + (1 - self.alpha) * self.prev_y
        return int(self.prev_x), int(self.prev_y)

smooth_mouse = SmoothMouse()

In [19]:
def find_finger_tip(processed):
    if processed.multi_hand_landmarks:
        hand_landmarks = processed.multi_hand_landmarks[0]
        return hand_landmarks.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP]
    
    return None


In [20]:
def move_mouse(index_finger_tip):
    if index_finger_tip is not None:
        x = int(index_finger_tip.x *screen_width)
        y = int(index_finger_tip.y *screen_width)

        pyautogui.moveTo(x,y)

In [21]:
def perform_click(gesture_type):
    if gesture_type == "left_click":
        pyautogui.click()
    elif gesture_type == "right_click":
        pyautogui.click(button='right')
    elif gesture_type == "double_click":
        pyautogui.doubleClick()

In [22]:

def detect_click_gestures(landmarks_list):
    if len(landmarks_list) >= 21:
        thumb_tip = landmarks_list[4]
        index_finger_tip = landmarks_list[8]
        middle_finger_tip = landmarks_list[12]

        thumb_index_dist = np.linalg.norm(np.array(thumb_tip) - np.array(index_finger_tip))
        thumb_middle_dist = np.linalg.norm(np.array(thumb_tip) - np.array(middle_finger_tip))

        # Left click gesture
        if thumb_index_dist < 0.03:
            perform_click("left_click")

        # Right click gesture
        elif thumb_middle_dist < 0.03:
            perform_click("right_click")

        # Double click gesture
        elif thumb_index_dist < 0.02:
            perform_click("double_click")

In [23]:
from multiprocessing import process
import mediapipe as mp

mpHands = mp.solutions.hands
mpDrawing = mp.solutions.drawing_utils
hands = mpHands.Hands()


def main():
    cap = cv2.VideoCapture(1)
    try:
        while cap .isOpened():
            ret,frame= cap.read()

            if not ret:
                break
            frame= cv2.flip(frame,1)
            frameRGB = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)

            processed = hands.process(frameRGB)


            landmarks_list = []

            if processed.multi_hand_landmarks:
                hand_landmarks = processed.multi_hand_landmarks[0]
                mpDrawing.draw_landmarks(frame,hand_landmarks,mpHands.HAND_CONNECTIONS)

                for lm in hand_landmarks.landmark:
                    landmarks_list.append((lm.x,lm.y))


                
                print(landmarks_list)
            
            detect_gestures(frame,landmarks_list,processed)
            
            cv2.imshow('Frame',frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    finally:
        cap.release()
        cv2.destroyAllWindows()

if __name__=='__main__':
    main()

[(0.15040311217308044, 0.42940956354141235), (0.16926506161689758, 0.45882344245910645), (0.1715322881937027, 0.5056779384613037), (0.16477981209754944, 0.5409314036369324), (0.15822833776474, 0.564788818359375), (0.16468676924705505, 0.514541506767273), (0.15966174006462097, 0.5626127123832703), (0.15649569034576416, 0.5906789302825928), (0.15479858219623566, 0.6107893586158752), (0.15183591842651367, 0.5145593881607056), (0.14789871871471405, 0.5628939270973206), (0.14751678705215454, 0.5912306308746338), (0.14844650030136108, 0.6147241592407227), (0.14041177928447723, 0.515337347984314), (0.13871191442012787, 0.5626177191734314), (0.1410163789987564, 0.5895253419876099), (0.14442138373851776, 0.6112155914306641), (0.13149228692054749, 0.5169352889060974), (0.13348719477653503, 0.556775689125061), (0.13765539228916168, 0.5778913497924805), (0.14146140217781067, 0.5948213934898376)]
[(0.15535470843315125, 0.42630863189697266), (0.16601091623306274, 0.45703497529029846), (0.16615635156